In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report, precision_recall_curve
from sklearn.impute import SimpleImputer


Import des données

In [41]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
print(f"train: {train_df.shape}, test: {test_df.shape}")


train: (225000, 325), test: (75000, 324)


Pré-traitement

In [42]:
# Suppression colonnes 100% NaN
cols_all_nan_train = train_df.columns[train_df.isna().mean() == 1.0]
train_df = train_df.drop(columns=cols_all_nan_train)
cols_all_nan_test = test_df.columns[test_df.isna().mean() == 1.0]
test_df = test_df.drop(columns=cols_all_nan_test)

On définit la target qu'on va essayer de prédire

In [43]:
target_col = train_df.columns[-1]

On conserve que les features qui sont bien corrélés avec la target

In [44]:
train_num = train_df.apply(pd.to_numeric, errors='coerce')
corr_with_target = train_num.corr(numeric_only=True)[target_col].drop(labels=[target_col], errors='ignore')
selected_features = corr_with_target[abs(corr_with_target) > 0.1].index.tolist()
print(f"{len(selected_features)} features sélectionnées")


23 features sélectionnées


In [45]:
X = train_df[selected_features].copy()
y = train_df[target_col]

encoder_dict = {}
for col in X.select_dtypes(include=['object', 'category']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    encoder_dict[col] = le

Création des datasets de train/eval

In [46]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
print(f"Split: {X_train.shape[0]} train / {X_val.shape[0]} val")

Split: 157500 train / 67500 val



Création de la pipeline de modèles

In [47]:
# Pipeline SVM avec imputation

pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    StandardScaler(),
    LinearSVC(C=1, class_weight='balanced', random_state=42)
)

Entraînement de la pipeline

In [48]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=1, class_weight='balanced', random_state=42))])

Evaluation du modèle

In [49]:
# Prédiction standard (pour référence)
y_pred_val = pipe.predict(X_val)
f1 = f1_score(y_val, y_pred_val)
print(f"F1 validation (predict): {f1:.4f}")
print(classification_report(y_val, y_pred_val))

# Recherche du meilleur seuil F1 en utilisant les scores (decision_function si disponible)
# On préfère decision_function pour LinearSVC (pas de predict_proba)
if hasattr(pipe, 'decision_function'):
    scores_val = pipe.decision_function(X_val)
else:
    try:
        scores_val = pipe.predict_proba(X_val)[:, 1]
    except Exception:
        # fallback: utiliser predictions (discret) si aucune score disponible
        scores_val = pipe.predict(X_val)

prec, recall, thresholds = precision_recall_curve(y_val, scores_val)
# thresholds has length = len(prec)-1; compute f1 for each threshold position
f1_scores = 2 * prec * recall / (prec + recall + 1e-12)
# best index among thresholds-related points: skip last prec/rec point (no threshold)
best_idx = f1_scores[:-1].argmax() if len(f1_scores) > 1 else 0
# ensure threshold exists (precision_recall_curve returns thresholds shorter by 1)
best_thresh = thresholds[best_idx] if len(thresholds) > 0 else 0.5
print(f"Meilleur seuil (val) pour F1: {best_thresh:.4f}")
print(f"F1 (au seuil): {f1_scores[best_idx]:.4f}")

# Prédiction avec seuil optimisé
y_pred_val_thresh = (scores_val >= best_thresh).astype(int)
print('F1 validation (après seuil):', f1_score(y_val, y_pred_val_thresh))
print(classification_report(y_val, y_pred_val_thresh))

F1 validation (predict): 0.3290
              precision    recall  f1-score   support

       False       0.97      0.71      0.82     61458
        True       0.21      0.79      0.33      6042

    accuracy                           0.71     67500
   macro avg       0.59      0.75      0.57     67500
weighted avg       0.90      0.71      0.77     67500

Meilleur seuil (val) pour F1: 0.2919
F1 (au seuil): 0.3660
F1 validation (après seuil): 0.36601523683338855
              precision    recall  f1-score   support

       False       0.95      0.86      0.90     61458
        True       0.27      0.55      0.37      6042

    accuracy                           0.83     67500
   macro avg       0.61      0.70      0.63     67500
weighted avg       0.89      0.83      0.85     67500



In [50]:
X_test = test_df.reindex(columns=selected_features, fill_value=np.nan).copy()
for col, le in encoder_dict.items():
    if col in X_test.columns:
        X_test[col] = X_test[col].astype(str)
        X_test[col] = X_test[col].map(lambda x: x if x in le.classes_ else None)
        X_test[col] = le.transform(X_test[col].fillna(le.classes_[0]))

# Prédictions sur le test en utilisant decision_function + meilleur seuil trouvé précédemment
if 'best_thresh' in globals():
    if hasattr(pipe, 'decision_function'):
        scores_test = pipe.decision_function(X_test)
    else:
        try:
            scores_test = pipe.predict_proba(X_test)[:, 1]
        except Exception:
            scores_test = pipe.predict(X_test)
    y_test_pred = (scores_test >= best_thresh).astype(int)
else:
    y_test_pred = pipe.predict(X_test)

id_col = 'ID' if 'ID' in test_df.columns else test_df.columns[0]

pred_df = pd.DataFrame({
    id_col: test_df[id_col],
    'pred': y_test_pred
})

pred_df.to_csv('predictions_svm.csv', index=False)
print('Exporté: predictions_svm.csv')
print(pred_df.head())


Exporté: predictions_svm.csv
       ID  pred
0  225000     0
1  225001     0
2  225002     0
3  225003     0
4  225004     0


Documentation / How to run
Ce notebook exécute les étapes suivantes :
1) Charger train/test depuis `data/`
2) Supprimer les colonnes 100% NaN
3) Sélectionner les features ayant |corr(Target)| > 0.1
4) Encodage des catégorielles avec LabelEncoder
5) Entraînement d'un SVM via GridSearchCV (scoring=f1)
6) Évaluation sur un split validation
7) Prédictions sur le jeu de test et export CSV

Notes:
- Ajuster `param_grid` pour plus d'itérations/hyperparamètres.
- Le SVM est standardisé avec `StandardScaler` pour de meilleures performances.
- Pour obtenir des probabilités, remplacer `probability=False` par `True` dans SVC.

print('Notebook prêt — exécutez les cellules dans l'ordre')
